In [ ]:
import pandas as pd
import json
import urllib.request
import sqlite3

In [2]:
# Copied from anki connect site
def request(action: str, **params) -> dict:
    return {"action": action, "params": params, "version": 6}


# Copied from anki connect site
def invoke(action: str, **params):
    requestJson = json.dumps(request(action, **params)).encode("utf-8")
    response = json.load(
        urllib.request.urlopen(
            urllib.request.Request("http://127.0.0.1:8765", requestJson)
        )
    )
    if len(response) != 2:
        raise Exception("response has an unexpected number of fields")
    if "error" not in response:
        raise Exception("response is missing required error field")
    if "result" not in response:
        raise Exception("response is missing required result field")
    if response["error"] is not None:
        raise Exception(response["error"])
    return response["result"]

In [11]:
db_path = "../data/clean/bir_warehouse.db"
query: str = """
SELECT y.year_name, ex.exam_type, q.question, qo.option_text, qo.is_correct
FROM questions_options AS qo
JOIN questions AS q ON qo.question_id = q.id
JOIN year AS y ON q.exam_year = y.id_year
JOIN exam AS ex ON q.exam_subject = ex.id_type;
"""

con = sqlite3.connect(db_path)

exams_df: pd.DataFrame = pd.read_sql(query,con)

con.close()

exams_df["index"] = exams_df["year_name"] + exams_df["exam_type"] + exams_df["question"]
exams_df["year_name"] = exams_df["year_name"].astype(int)
exams_df["is_correct"] = exams_df["is_correct"].astype(bool)

exams_df.head()

,year_name,exam_type,question,option_text,is_correct,index
0,2024,bir,1. La barrera hematoencefálica:,1. Es permeable a todas las sustancias present...,False,2024bir1. La barrera hematoencefálica:
1,2024,bir,1. La barrera hematoencefálica:,2. Es permeable al O2 y al CO2.,True,2024bir1. La barrera hematoencefálica:
2,2024,bir,1. La barrera hematoencefálica:,3. Es impermeable al etanol.,False,2024bir1. La barrera hematoencefálica:
3,2024,bir,1. La barrera hematoencefálica:,4. Es impermeable al agua.,False,2024bir1. La barrera hematoencefálica:
4,2024,bir,2. El espacio subaracnoideo se encuentra:,1. Entre la aracnoides y la duramadre.,False,2024bir2. El espacio subaracnoideo se encuentra:


In [12]:
exams_df_group = exams_df.groupby("index")
exams_df_group.head()

,year_name,exam_type,question,option_text,is_correct,index
0,2024,bir,1. La barrera hematoencefálica:,1. Es permeable a todas las sustancias present...,False,2024bir1. La barrera hematoencefálica:
1,2024,bir,1. La barrera hematoencefálica:,2. Es permeable al O2 y al CO2.,True,2024bir1. La barrera hematoencefálica:
2,2024,bir,1. La barrera hematoencefálica:,3. Es impermeable al etanol.,False,2024bir1. La barrera hematoencefálica:
3,2024,bir,1. La barrera hematoencefálica:,4. Es impermeable al agua.,False,2024bir1. La barrera hematoencefálica:
4,2024,bir,2. El espacio subaracnoideo se encuentra:,1. Entre la aracnoides y la duramadre.,False,2024bir2. El espacio subaracnoideo se encuentra:
...,...,...,...,...,...,...
24280,2013,bir,235. En la electroforesis de hemoglobina en ac...,1. Se emplean como colorantes el Rojo Neutro o...,False,2013bir235. En la electroforesis de hemoglobin...
24281,2013,bir,235. En la electroforesis de hemoglobina en ac...,2. Se obtienen tres fraccione s diferenciadas:...,True,2013bir235. En la electroforesis de hemoglobin...
24282,2013,bir,235. En la electroforesis de hemoglobina en ac...,3. La fracción mayoritaria es la HbA2.,False,2013bir235. En la electroforesis de hemoglobin...
24283,2013,bir,235. En la electroforesis de hemoglobina en ac...,4. Se obtiene el hemolizado empleando CINH4.,False,2013bir235. En la electroforesis de hemoglobin...


In [ ]:
for _, group in exams_df_group:
    ...